# Creating Base DF's

In [1]:
# Library Calls
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_columns = None

# Create Functions for Processing

In [2]:
# Change Columns Names (Not Currently Used)
def rename_columns(df):
    df = df.iloc[:,3:]
    df.columns = list(range(df.shape[1]))
    return df

In [3]:
# Subset the DataFrame
def subset_data(df):
    # Create a DataFrame of all First time, Full time students for the Fall cohort
    first_full = df[(df.TIME_STATUS == 'UG Full-Time') & (df.STUDENT_TYPE == 'First-Time Freshman')]

    # Create a DataFrame of all First time, Part time students for the Fall cohort
    first_part = df[(df.TIME_STATUS == 'UG Part-Time') & (df.STUDENT_TYPE == 'First-Time Freshman')]

    # Create a DataFrame of all Transfer, Full time students for the Fall cohort
    transfer_full = df[(df.TIME_STATUS == 'UG Full-Time') & (df.STUDENT_TYPE == 'Transfer Student')]

    # Create a DataFrame of all Transfer, Part time students for the Fall cohort
    transfer_part = df[(df.TIME_STATUS == 'UG Part-Time') & (df.STUDENT_TYPE == 'Transfer Student')]
    
    return first_full, first_part, transfer_full, transfer_part

In [4]:
# Inspect DataFrame for size and percentages of different groups
def get_df_counts(df,name):
    
    Number_first_full = df[(df.TIME_STATUS == 'UG Full-Time') & (df.STUDENT_TYPE == 'First-Time Freshman')].shape[0]

    Number_first_part = df[(df.TIME_STATUS == 'UG Part-Time') & (df.STUDENT_TYPE == 'First-Time Freshman')].shape[0]

    Number_transfer_full = df[(df.TIME_STATUS == 'UG Full-Time') & (df.STUDENT_TYPE == 'Transfer Student')].shape[0]

    Number_transfer_part = df[(df.TIME_STATUS == 'UG Part-Time') & (df.STUDENT_TYPE == 'Transfer Student')].shape[0]
    
    (print(f'{name[:-4]} Number of First Time Full Time Students: {Number_first_full}'))
             # {name[:-4]} Number of First TIme Part Time Students: {Number_first_part}\n\
             # {name[:-4]} Number of Transfer Full TIme Students: {Number_transfer_full},\n\
             # {name[:-4]} Number of Transfer Part Time Students: {Number_transfer_part}"))

In [5]:
# Assign states with credits as split. 
# IE freshman_1 == 0 - 15 credits, 
    #freshman_2 = 16 - 30 credits, 
    #sophomore_1 == 31 - 45 credits, 
    #..... , 
    #senior_2 == 106 - 119,
    #graduate
def assign_credit_based_states(df,df_name):
    """
    This Function takes in a dataframe of student starting credits per semester and assigns them a state based on the amount of credits they have.
    """
    
    # Bring in the overall df to assign Transferred, Dropped, Graduated States
    all_cohort = pd.read_csv('all_ftf_cohorts.csv')
    # Make the students number the index
    all_cohort = all_cohort.set_index('pid')
    
    # Iterate through the columns
    for col in df.columns:
        # Skip columns that are not relavant
        if col in ['PIDM','STUDENT_TYPE','TIME_STATUS']:
            continue
        # First Column should be the same as the file name and shows the first semester at the school. 
        elif col == df_name[:6]:
            df[col] = f'Freshman_1'
        # Now check the rows in each column
        else:
            for row in df[col].index:
                # Check to see if the student is in the overall dataframe that contains all first time full time students
                ############### Note all_cohort only contains first time full time students and the following will only run for the first time full time students subset ###########################
                if row in all_cohort.index:
                    # Check to see if they have a Transferred Out status in the overall dataframe and if so then assign that value in the new df
                    if all_cohort[col][row] == "Transferred Out":
                        df[col][row] = "Transferred_Out"
                    # Check to see if they have a Dropped Out status in the overall dataframe and if so then assign that value in the new df
                    elif all_cohort[col][row] == 'Dropped Out':
                        df[col][row] = 'Dropped_Out'
                    # Check to see if they have a Graduated status in the overall dataframe and if so then assign that value in the new df
                    elif all_cohort[col][row] == 'Graduated':
                        df[col][row] = 'Graduated'
                    # Check if there is a Nan value and if so this indicates the student took a sabbatical if the above values arent present
                    elif np.isnan(df[col][row]):
                        df[col][row] = f'Sabbatical'
                    # If none of the above cases happened we assume there is a credit value and assign what state a student is based on credit amount
                    elif (df[col][row] >= 0) & (df[col][row] <= 15):
                        df[col][row] = f'Freshman_1'
                    elif (df[col][row] > 15) & (df[col][row] <= 30):
                        df[col][row] = f'Freshman_2'
                    elif (df[col][row] > 30) & (df[col][row] <= 45):
                        df[col][row] = f'Sophomore_1'
                    elif (df[col][row] > 45) & (df[col][row] <= 60):
                        df[col][row] = f'Sophomore_2'
                    elif (df[col][row] > 60) & (df[col][row] <= 75):
                        df[col][row] = f'Junior_1'
                    elif (df[col][row] > 75) & (df[col][row] <= 90):
                        df[col][row] = f'Junior_2'
                    elif (df[col][row] > 90) & (df[col][row] <= 105):
                        df[col][row] = f'Senior_1'
                    elif (df[col][row] > 105) & (df[col][row] <= 120):
                        df[col][row] = f'Senior_2'
        for row in df[col].index:
            if col[4:6] == '30':
                df[col][row] = f'Spring_{df[col][row]}'
            elif col[4:6] == '40':
                df[col][row] = f'Summer_{df[col][row]}'
            elif col[4:6] == '50':
                df[col][row] = f'Fall_{df[col][row]}'
            else:
                continue
    return df

In [6]:
def credit_wrangle_df(string_csv_name):
        # Using the string nmae of csv file we will read in the dataframe
        df = pd.read_csv(string_csv_name)
        df = df.set_index('PIDM')
        
        # Print out proportions
        print(f'Credit Based')
        print(get_df_counts(df,string_csv_name)) 
        
        # Subset the dataframe into sub groups of student types
        first_full, first_part, transfer_full, transfer_part = subset_data(df)
        
        # reassign data to states
        first_full = assign_credit_based_states(first_full, string_csv_name)
        first_part = assign_credit_based_states(first_part, string_csv_name)
        transfer_full= assign_credit_based_states(transfer_full, string_csv_name)
        transfer_part= assign_credit_based_states(transfer_part, string_csv_name)

        return first_full, first_part, transfer_full, transfer_part

In [7]:
# Create semester seperated dataframe
# If first semester as class = class_1
# If second to n semester as class = class_n

def assign_semester_based_states(df, df_name):
    """
    This Function takes in a dataframe of student starting credits per semester and assigns them a state based on semesters in a class.
    """
    
    # Bring in the overall df to assign Transferred, Dropped, Graduated States
    all_cohort = pd.read_csv('all_ftf_cohorts.csv')
    # Make the students number the index
    all_cohort = all_cohort.set_index('pid')
    
    # Iterate through the columns
    for col in df.columns:
        # Skip columns that are not relavant
        if col in ['PIDM','STUDENT_TYPE','TIME_STATUS']:
            continue
         # First Column should be the same as the file name and shows the first semester at the school. 
        elif col == df_name[:6]:
            df[col] = f'Fall_Freshman_1'
        # Now check the rows in each column
        else:
            for row in df[col].index:
                # Check to see if the student is in the overall dataframe that contains all first time full time students
                ############### Note all_cohort only contains first time full time students and the following will only run for the first time full time students subset ###########################
                if row in all_cohort.index:
                    # Check to see if they have a Transferred Out status in the overall dataframe and if so then assign that value in the new df
                    if all_cohort[col][row] == "Transferred Out":
                        df[col][row] = "Transferred_Out"
                    # Check to see if they have a Dropped Out status in the overall dataframe and if so then assign that value in the new df
                    elif all_cohort[col][row] == 'Dropped Out':
                        df[col][row] = 'Dropped_Out'
                    # Check to see if they have a Graduated status in the overall dataframe and if so then assign that value in the new df
                    elif all_cohort[col][row] == 'Graduated':
                        df[col][row] = 'Graduated'
                    # Check if there is a Nan value and if so this indicates the student took a sabbatical if the above values arent present
                    elif np.isnan(df[col][row]):
                        df[col][row] = f'Sabbatical'
                    # If none of the above cases happened we assume there is a credit value and assign what state a student is based on credit
                    # amount and time in class
                    elif (df[col][row] >= 0) & (df[col][row] <= 30):
                        df[col][row] = f'Freshman_n'
                        
                    elif (df[col][row] > 30) & (df[col][row] <= 60):
                        if f'Sophomore_1' in df[df.index == row].values:
                            df[col][row] = f'Sophomore_n'
                        else:
                            df[col][row] = f'Sophomore_1'
                        
                    elif (df[col][row] > 60) & (df[col][row] <= 90):
                        if f'Junior_1' in df[df.index == row].values:
                            df[col][row] = f'Junior_n'
                        else:
                            df[col][row] = f'Junior_1'
                        
                    elif (df[col][row] > 90) & (df[col][row] <= 120):
                        if f'Senior_1' in df[df.index == row].values:
                            df[col][row] = f'Senior_n'
                        else: 
                            df[col][row] = f'Senior_1'
        for row in df[col].index:
            if col[4:6] == '30':
                df[col][row] = f'Spring_{df[col][row]}'
            elif col[4:6] == '40':
                df[col][row] = f'Summer_{df[col][row]}'
            elif col[4:6] == '50':
                df[col][row] = f'Fall_{df[col][row]}'
            else:
                continue
    return df

In [8]:
def semester_wrangle_df(string_csv_name):
        # Using the string nmae of csv file we will read in the dataframe
        df = pd.read_csv(string_csv_name)
        df = df.set_index('PIDM')
        
        # Print out proportions
        print(f'Semester Based')
        print(get_df_counts(df,string_csv_name)) 
        
        # Subset the dataframe into sub groups of student types
        first_full, first_part, transfer_full, transfer_part = subset_data(df)
        
        # reassign data to states
        first_full = assign_semester_based_states(first_full, string_csv_name)
        first_part = assign_semester_based_states(first_part, string_csv_name)
        transfer_full= assign_semester_based_states(transfer_full, string_csv_name)
        transfer_part= assign_semester_based_states(transfer_part, string_csv_name)

        return first_full, first_part, transfer_full, transfer_part

In [9]:
def combine_all_df():
    credit_df_2011_2018 = pd.DataFrame()
    semester_df_2011_2018 = pd.DataFrame()
    list_file_names = ['201150_cohort.csv','201250_cohort.csv','201350_cohort.csv','201450_cohort.csv','201550_cohort.csv','201650_cohort.csv','201750_cohort.csv','201850_cohort.csv']
    for cohort in list_file_names:
        cff, cfp, ctf, ctp = credit_wrangle_df(cohort)
        sff, sfp, stf, stp = semester_wrangle_df(cohort)
        credit_df_2011_2018 = pd.concat([credit_df_2011_2018, cff], axis=0)
        semester_df_2011_2018 = pd.concat([semester_df_2011_2018, sff], axis=0)
    print(f'Total Credit Count: {credit_df_2011_2018.shape}')
    print(f'Total Semester Size: {semester_df_2011_2018.shape}')
    return credit_df_2011_2018.iloc[:,2:], semester_df_2011_2018.iloc[:,2:]

# Overall Dataframes

In [10]:
list_file_names = ['201150_cohort.csv','201250_cohort.csv','201350_cohort.csv','201450_cohort.csv','201550_cohort.csv','201650_cohort.csv','201750_cohort.csv','201850_cohort.csv']
credit_based, semester_based = combine_all_df()

Credit Based
201150_cohort Number of First Time Full Time Students: 1691
None
Semester Based
201150_cohort Number of First Time Full Time Students: 1691
None
Credit Based
201250_cohort Number of First Time Full Time Students: 1621
None
Semester Based
201250_cohort Number of First Time Full Time Students: 1621
None
Credit Based
201350_cohort Number of First Time Full Time Students: 1624
None
Semester Based
201350_cohort Number of First Time Full Time Students: 1624
None
Credit Based
201450_cohort Number of First Time Full Time Students: 1688
None
Semester Based
201450_cohort Number of First Time Full Time Students: 1688
None
Credit Based
201550_cohort Number of First Time Full Time Students: 1569
None
Semester Based
201550_cohort Number of First Time Full Time Students: 1569
None
Credit Based
201650_cohort Number of First Time Full Time Students: 1881
None
Semester Based
201650_cohort Number of First Time Full Time Students: 1881
None
Credit Based
201750_cohort Number of First Time Full

In [11]:
credit_based.iloc[:,2:]

,201240,201250,201330,201340,201350,201430,201440,201450,201530,201540,201550,201630,201640,201650,201730,201740,201750,201830,201840,201850,201930,201940,201950,202030,202040,202050,202130,202140,202150,202230
PIDM,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
94308,Summer_Sabbatical,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,NaN,NaN
260370,Summer_Sabbatical,Fall_Freshman_1,Spring_Sabbatical,Summer_Sabbatical,Fall_Sabbatical,Spring_Sabbatical,Summer_Sabbatical,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,NaN,NaN
316154,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,NaN,NaN
345391,Summer_Sabbatical,Fall_Freshman_1,Spring_Freshman_2,Summer_Sabbatical,Fall_Sophomore_1,Spring_Sophomore_2,Summer_Sabbatical,Fall_Junior_1,Spring_Junior_2,Summer_Sabbatical,Fall_Junior_2,Spring_Senior_1,Summer_Graduated,Fall_Graduated,Spring_Graduated,Summer_Graduated,Fall_Graduated,Spring_Graduated,Summer_Graduated,Fall_Graduated,Spring_Graduated,Summer_Graduated,Fall_Graduated,Spring_Graduated,Summer_Graduated,Fall_Graduated,Spring_Graduated,Summer_Graduated,NaN,NaN
373354,Summer_Sabbatical,Fall_Sabbatical,Spring_Sabbatical,Summer_Sabbatical,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1121579,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fall_Freshman_1,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out
1121946,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fall_Freshman_1,Spring_Freshman_2,Summer_Sophomore_1,Fall_Sophomore_1,Spring_Sophomore_1,Summer_Sabbatical,Fall_Sabbatical,Spring_Sophomore_2,Summer_Sabbatical,Fall_Sophomore_2,Spring_Junior_1
1121950,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fall_Freshman_1,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_D

In [12]:
semester_based

,201150,201230,201240,201250,201330,201340,201350,201430,201440,201450,201530,201540,201550,201630,201640,201650,201730,201740,201750,201830,201840,201850,201930,201940,201950,202030,202040,202050,202130,202140,202150,202230
PIDM,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
94308,Fall_Fall_Freshman_1,Spring_Freshman_n,Summer_Sabbatical,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,NaN,NaN
260370,Fall_Fall_Freshman_1,Spring_Freshman_n,Summer_Sabbatical,Fall_Freshman_n,Spring_Sabbatical,Summer_Sabbatical,Fall_Sabbatical,Spring_Sabbatical,Summer_Sabbatical,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,NaN,NaN
316154,Fall_Fall_Freshman_1,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,NaN,NaN
345391,Fall_Fall_Freshman_1,Spring_Freshman_n,Summer_Sabbatical,Fall_Freshman_n,Spring_Freshman_n,Summer_Sabbatical,Fall_Sophomore_1,Spring_Sophomore_1,Summer_Sabbatical,Fall_Junior_1,Spring_Junior_1,Summer_Sabbatical,Fall_Junior_1,Spring_Senior_1,Summer_Graduated,Fall_Graduated,Spring_Graduated,Summer_Graduated,Fall_Graduated,Spring_Graduated,Summer_Graduated,Fall_Graduated,Spring_Graduated,Summer_Graduated,Fall_Graduated,Spring_Graduated,Summer_Graduated,Fall_Graduated,Spring_Graduated,Summer_Graduated,NaN,NaN
373354,Fall_Fall_Freshman_1,Spring_Freshman_n,Summer_Sabbatical,Fall_Sabbatical,Spring_Sabbatical,Summer_Sabbatical,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1121579,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fall_Fall_Freshman_1,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out
1121946,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fall_Fall_Freshman_1,Spring_Freshman_n,Summer_Sophomore_1,Fall_Sophomore_1,Spring_Sophomore_1,Summer_Sabbatical,Fall_Sabbatical,Spring_Sophomore_1,Summ

# Fall 2011 Cohort

### Credit Based States

In [13]:
credit_first_full_2011, credit_first_part_2011, credit_transfer_full_2011, credit_transfer_part_2011 = credit_wrangle_df('201150_cohort.csv')

Credit Based
201150_cohort Number of First Time Full Time Students: 1691
None


In [14]:
credit_first_full_2011.head()

,STUDENT_TYPE,TIME_STATUS,201150,201230,201240,201250,201330,201340,201350,201430,201440,201450,201530,201540,201550,201630,201640,201650,201730,201740,201750,201830,201840,201850,201930,201940,201950,202030,202040,202050,202130,202140
PIDM,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
94308,First-Time Freshman,UG Full-Time,Fall_Freshman_1,Spring_Freshman_1,Summer_Sabbatical,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out
260370,First-Time Freshman,UG Full-Time,Fall_Freshman_1,Spring_Freshman_1,Summer_Sabbatical,Fall_Freshman_1,Spring_Sabbatical,Summer_Sabbatical,Fall_Sabbatical,Spring_Sabbatical,Summer_Sabbatical,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out
316154,First-Time Freshman,UG Full-Time,Fall_Freshman_1,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out
345391,First-Time Freshman,UG Full-Time,Fall_Freshman_1,Spring_Freshman_1,Summer_Sabbatical,Fall_Freshman_1,Spring_Freshman_2,Summer_Sabbatical,Fall_Sophomore_1,Spring_Sophomore_2,Summer_Sabbatical,Fall_Junior_1,Spring_Junior_2,Summer_Sabbatical,Fall_Junior_2,Spring_Senior_1,Summer_Graduated,Fall_Graduated,Spring_Graduated,Summer_Graduated,Fall_Graduated,Spring_Graduated,Summer_Graduated,Fall_Graduated,Spring_Graduated,Summer_Graduated,Fall_Graduated,Spring_Graduated,Summer_Graduated,Fall_Graduated,Spring_Graduated,Summer_Graduated
373354,First-Time Freshman,UG Full-Time,Fall_Freshman_1,Spring_Freshman_1,Summer_Sabbatical,Fall_Sabbatical,Spring_Sabbatical,Summer_Sabbatical,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out


In [15]:
credit_first_full_2011['201230'].value_counts()

Spring_Freshman_1         1023
Spring_Dropped_Out         230
Spring_Freshman_2          220
Spring_Sabbatical          119
Spring_Transferred_Out      48
Spring_Sophomore_1          34
Spring_Sophomore_2          10
Spring_Junior_1              5
Spring_Junior_2              2
Name: 201230, dtype: int64

### Semester Based States

In [16]:
semester_first_full_2011, semester_first_part_2011, semester_transfer_full_2011, semester_transfer_part_2011 = semester_wrangle_df('201150_cohort.csv')

Semester Based
201150_cohort Number of First Time Full Time Students: 1691
None


In [17]:
semester_first_full_2011.head()

,STUDENT_TYPE,TIME_STATUS,201150,201230,201240,201250,201330,201340,201350,201430,201440,201450,201530,201540,201550,201630,201640,201650,201730,201740,201750,201830,201840,201850,201930,201940,201950,202030,202040,202050,202130,202140
PIDM,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
94308,First-Time Freshman,UG Full-Time,Fall_Fall_Freshman_1,Spring_Freshman_n,Summer_Sabbatical,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out
260370,First-Time Freshman,UG Full-Time,Fall_Fall_Freshman_1,Spring_Freshman_n,Summer_Sabbatical,Fall_Freshman_n,Spring_Sabbatical,Summer_Sabbatical,Fall_Sabbatical,Spring_Sabbatical,Summer_Sabbatical,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out
316154,First-Time Freshman,UG Full-Time,Fall_Fall_Freshman_1,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out
345391,First-Time Freshman,UG Full-Time,Fall_Fall_Freshman_1,Spring_Freshman_n,Summer_Sabbatical,Fall_Freshman_n,Spring_Freshman_n,Summer_Sabbatical,Fall_Sophomore_1,Spring_Sophomore_1,Summer_Sabbatical,Fall_Junior_1,Spring_Junior_1,Summer_Sabbatical,Fall_Junior_1,Spring_Senior_1,Summer_Graduated,Fall_Graduated,Spring_Graduated,Summer_Graduated,Fall_Graduated,Spring_Graduated,Summer_Graduated,Fall_Graduated,Spring_Graduated,Summer_Graduated,Fall_Graduated,Spring_Graduated,Summer_Graduated,Fall_Graduated,Spring_Graduated,Summer_Graduated
373354,First-Time Freshman,UG Full-Time,Fall_Fall_Freshman_1,Spring_Freshman_n,Summer_Sabbatical,Fall_Sabbatical,Spring_Sabbatical,Summer_Sabbatical,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out


# Fall 2012 Cohort

### Credit Based States

In [18]:
credit_first_full_2012, credit_first_part_2012, credit_transfer_full_2012, credit_transfer_part_2012 = credit_wrangle_df('201250_cohort.csv')

Credit Based
201250_cohort Number of First Time Full Time Students: 1621
None


In [19]:
credit_first_full_2012.head()

,STUDENT_TYPE,TIME_STATUS,201250,201330,201340,201350,201430,201440,201450,201530,201540,201550,201630,201640,201650,201730,201740,201750,201830,201840,201850,201930,201940,201950,202030,202040,202050,202130,202140,202150,202230
PIDM,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
28419,First-Time Freshman,UG Full-Time,Fall_Freshman_1,Spring_Freshman_2,Summer_Sabbatical,Fall_Sophomore_1,Spring_Sophomore_2,Summer_Junior_1,Fall_Junior_1,Spring_Junior_2,Summer_Sabbatical,Fall_Senior_2,Spring_Senior_2,Summer_Sabbatical,Fall_Graduated,Spring_Graduated,Summer_Graduated,Fall_Graduated,Spring_Graduated,Summer_Graduated,Fall_Graduated,Spring_Graduated,Summer_Graduated,Fall_Graduated,Spring_Graduated,Summer_Graduated,Fall_Graduated,Spring_Graduated,Summer_Graduated,Fall_Graduated,Spring_Graduated
437489,First-Time Freshman,UG Full-Time,Fall_Freshman_1,Spring_Freshman_1,Summer_Sabbatical,Fall_Sabbatical,Spring_Sabbatical,Summer_Sabbatical,Fall_Sabbatical,Spring_Sabbatical,Summer_Sabbatical,Fall_Sabbatical,Spring_Sabbatical,Summer_Sabbatical,Fall_Sabbatical,Spring_Sabbatical,Summer_Sabbatical,Fall_Sabbatical,Spring_Sabbatical,Summer_Sabbatical,Fall_Sabbatical,Spring_Sabbatical,Summer_Sabbatical,Fall_Sabbatical,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out
466834,First-Time Freshman,UG Full-Time,Fall_Freshman_1,Spring_Sabbatical,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out
508374,First-Time Freshman,UG Full-Time,Fall_Freshman_1,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out
560328,First-Time Freshman,UG Full-Time,Fall_Freshman_1,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out


### Semester Bases States

In [20]:
semester_first_full_2012, semester_first_part_2012, semester_transfer_full_2012, semester_transfer_part_2012 = semester_wrangle_df('201250_cohort.csv')

Semester Based
201250_cohort Number of First Time Full Time Students: 1621
None


In [21]:
semester_first_full_2012.head()

,STUDENT_TYPE,TIME_STATUS,201250,201330,201340,201350,201430,201440,201450,201530,201540,201550,201630,201640,201650,201730,201740,201750,201830,201840,201850,201930,201940,201950,202030,202040,202050,202130,202140,202150,202230
PIDM,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
28419,First-Time Freshman,UG Full-Time,Fall_Fall_Freshman_1,Spring_Freshman_n,Summer_Sabbatical,Fall_Sophomore_1,Spring_Sophomore_1,Summer_Junior_1,Fall_Junior_1,Spring_Junior_1,Summer_Sabbatical,Fall_Senior_1,Spring_Senior_1,Summer_Sabbatical,Fall_Graduated,Spring_Graduated,Summer_Graduated,Fall_Graduated,Spring_Graduated,Summer_Graduated,Fall_Graduated,Spring_Graduated,Summer_Graduated,Fall_Graduated,Spring_Graduated,Summer_Graduated,Fall_Graduated,Spring_Graduated,Summer_Graduated,Fall_Graduated,Spring_Graduated
437489,First-Time Freshman,UG Full-Time,Fall_Fall_Freshman_1,Spring_Freshman_n,Summer_Sabbatical,Fall_Sabbatical,Spring_Sabbatical,Summer_Sabbatical,Fall_Sabbatical,Spring_Sabbatical,Summer_Sabbatical,Fall_Sabbatical,Spring_Sabbatical,Summer_Sabbatical,Fall_Sabbatical,Spring_Sabbatical,Summer_Sabbatical,Fall_Sabbatical,Spring_Sabbatical,Summer_Sabbatical,Fall_Sabbatical,Spring_Sabbatical,Summer_Sabbatical,Fall_Sabbatical,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out
466834,First-Time Freshman,UG Full-Time,Fall_Fall_Freshman_1,Spring_Sabbatical,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out,Summer_Transferred_Out,Fall_Transferred_Out,Spring_Transferred_Out
508374,First-Time Freshman,UG Full-Time,Fall_Fall_Freshman_1,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out
560328,First-Time Freshman,UG Full-Time,Fall_Fall_Freshman_1,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out


# Fall 2013 Cohort

### Credit Based States

In [22]:
credit_first_full_2013, credit_first_part_2013, credit_transfer_full_2013, credit_transfer_part_2013 = credit_wrangle_df('201350_cohort.csv')

Credit Based
201350_cohort Number of First Time Full Time Students: 1624
None


In [23]:
credit_first_full_2013.head()

,STUDENT_TYPE,TIME_STATUS,201350,201430,201440,201450,201530,201540,201550,201630,201640,201650,201730,201740,201750,201830,201840,201850,201930,201940,201950,202030,202040,202050,202130,202140,202150,202230
PIDM,,,,,,,,,,,,,,,,,,,,,,,,,,,,
64583,First-Time Freshman,UG Full-Time,Fall_Freshman_1,Spring_Freshman_1,Summer_Sabbatical,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out
470554,First-Time Freshman,UG Full-Time,Fall_Freshman_1,Spring_Freshman_1,Summer_Sabbatical,Fall_Freshman_2,Spring_Sophomore_1,Summer_Sabbatical,Fall_Sophomore_2,Spring_Junior_1,Summer_Transferred_Out,Fall_Junior_2,Spring_Senior_1,Summer_Senior_2,Fall_Graduated,Spring_Graduated,Summer_Graduated,Fall_Graduated,Spring_Graduated,Summer_Graduated,Fall_Graduated,Spring_Graduated,Summer_Graduated,Fall_Graduated,Spring_Graduated,Summer_Graduated,Fall_Graduated,Spring_Graduated
549073,First-Time Freshman,UG Full-Time,Fall_Freshman_1,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out,Summer_Dropped_Out,Fall_Dropped_Out,Spring_Dropped_Out
555516,First-Time Freshman,UG Full-Time,Fall_Freshman_1,Spring_Freshman_1,Summer_Sabbatical,Fall_Freshman_2,Spring_Sophomore_1,Summer_Sophomore_2,Fall_Transferred_Out,Spring_Junior_1,Summer_Sabbatical,Fall_Junior_2,Spring_Junior_2,Summer_Sabbatical,Fall_Senior_1,Spring_Senior_2,Summer_Senior_2,Fall_125.0,Spring_Graduated,Summer_Graduated,Fall_Graduated,Spring_Graduated,Summer_Graduated,Fall_Graduated,Spring_Graduated,Summer_Graduated,Fall_Graduated,Spring_Graduated
594061,First-Time Freshman,UG Full-Time,Fall_Freshman_1,Spring_Sophomore_1,Summer_Sabbatical,Fall_Sophomore_2,Spring_Junior_1,Summer_Junior_2,Fall_Junior_2,Spring_Senior_1,Summer_Senior_2,Fall_121.0,Spring_133.0,Summer_Sabbatical,Fall_Graduated,Spring_Graduated,Summer_Graduated,Fall_Graduated,Spring_Graduated,Summer_Graduated,Fall_Graduated,Spring_Graduated,Summer_Graduated,Fall_Graduated,Spring_Graduated,Summer_Graduated,Fall_Graduated,Spring_Graduated


### Semester Based States

In [ ]:
semester_first_full_2013, semester_first_part_2013, semester_transfer_full_2013, semester_transfer_part_2013 = semester_wrangle_df('201350_cohort.csv')

Semester Based
201350_cohort Number of First Time Full Time Students: 1624
None


In [ ]:
semester_first_full_2013.head()

# Fall 2014 Cohort

### Credit Based States

In [ ]:
credit_first_full_2014, credit_first_part_2014, credit_transfer_full_2014, credit_transfer_part_2014 = credit_wrangle_df('201450_cohort.csv')

In [ ]:
credit_first_full_2014.head()

### Semester Based States

In [ ]:
semester_first_full_2014, semester_first_part_2014, semester_transfer_full_2014, semester_transfer_part_2014 = semester_wrangle_df('201450_cohort.csv')

In [ ]:
semester_first_full_2014.head()

# Fall 2015 Cohort

### Credit Based States

In [ ]:
credit_first_full_2015, credit_first_part_2015, credit_transfer_full_2015, credit_transfer_part_2015 = credit_wrangle_df('201550_cohort.csv')

In [ ]:
credit_first_full_2015.head()

### Semester Based States

In [ ]:
semester_first_full_2015, semester_first_part_2015, semester_transfer_full_2015, semester_transfer_part_2015 = semester_wrangle_df('201550_cohort.csv')

In [ ]:
semester_first_full_2015.head()

# Fall 2016 Cohort

### Credit Base States

In [ ]:
credit_first_full_2016, credit_first_part_2016, credit_transfer_full_2016, credit_transfer_part_2016 = credit_wrangle_df('201650_cohort.csv')

In [ ]:
credit_first_full_2016.head()

### Semester Based States

In [ ]:
semester_first_full_2016, semester_first_part_2016, semester_transfer_full_2016, semester_transfer_part_2016 = semester_wrangle_df('201650_cohort.csv')

In [ ]:
semester_first_full_2016.head()

# Fall 2017 Cohort

### Credit Based States

In [ ]:
credit_first_full_2017, credit_first_part_2017, credit_transfer_full_2017, credit_transfer_part_2017 = credit_wrangle_df('201750_cohort.csv')

In [ ]:
credit_first_full_2017.head()

### Semester Based States

In [ ]:
semester_first_full_2017, semester_first_part_2017, semester_transfer_full_2017, semester_transfer_part_2017 = semester_wrangle_df('201750_cohort.csv')

In [ ]:
semester_first_full_2017.head()

# Fall 2018 Cohort

### Credit Based States

In [ ]:
credit_first_full_2018, credit_first_part_2018, credit_transfer_full_2018, credit_transfer_part_2018 = credit_wrangle_df('201850_cohort.csv')

In [ ]:
credit_first_full_2018.head()